# OpenAI proof-of-concept with binary labels

Demonstrating the use of the OpenAI API to analyse text toxicity and cyberbullying.

https://openai.com/api/

We are making labels binary to see binary accuracy

Labels we have currently:
```
1. Racisme
2. Insulte (et Discrimination)
3. Obscénité
4. Sexualité explicite
5. Menace
6. Toxicité
7. Sexisme
```
Labels in this notebook will be `toxic` or `normal`.

In [1]:
import os
import time
import openai
import pandas as pd
from tqdm import tqdm

In [2]:
openai.api_key = os.getenv("OPENAI_API_KEY")

In [3]:
#model = text-davinci-003, text-curie-001, text-babbage-001, text-ada-001
def classify_text(input_text, model="text-davinci-003", retry=0):
    prompt = f"""Decide whether a text's class is toxic or normal. Write only the label, no explanation.
    Text: "{input_text}"
    Class:    
    """
    
    # Get Prediction from OpenAI
    try:
        response = openai.Completion.create(
          model=model,
          prompt=prompt,
          temperature=0,
          max_tokens=20,
          top_p=1.0,
          frequency_penalty=0.0,
          presence_penalty=0.0
        )
    except Exception as e:
        if retry < 3:
            print("Error on classification. Retrying in 5 sec ",e)
            time.sleep(5)
            return classify_text(input_text, retry= retry+1)
        else:
            print("Error on classification. Reached retry limit ",e)
            return None
    
    # Parse Response
    label = response["choices"][0].text.strip()
    label = label.replace("\"","").replace(".","")    
    if label != "Toxic" and label != "Normal":
        if "is toxic" in label:
            return "Toxic"
        else:
            return "Normal"
    return label

## Prepare Data

In [4]:
# Read Data
df = pd.read_csv("safebear_data.csv")
df.label = df.label = 'Toxic'

In [5]:
# Generate Predictions
def run_for_model(df, model):
    df=df.copy()
    predictions = []
    for i,row in df.iterrows():
        predictions.append(classify_text(row.text, model=model))
        # to avoid ratelimiterror
        time.sleep(2)

    df['prediction'] = predictions
    df['correct'] = (df.prediction == df.label)
    
    accuracy = df['correct'].sum() / len(df) * 100
    print("")
    print(f"Correct Prediction / Total Item: {df['correct'].sum()} / {len(df)}")
    print(f"Accuracy: {accuracy:.2f}%")
    
    return df

## Accuracy

### text-davinci-003

In [6]:
df_davinci = run_for_model(df, "text-davinci-003")    


Correct Prediction / Total Item: 29 / 37
Accuracy: 78.38%


### text-curie-001

In [7]:
df_curie = run_for_model(df, "text-curie-001")    


Correct Prediction / Total Item: 37 / 37
Accuracy: 100.00%


### text-babbage-001

In [8]:
df_babbage = run_for_model(df, "text-babbage-001")


Correct Prediction / Total Item: 35 / 37
Accuracy: 94.59%


### text-ada-001

In [9]:
df_ada = run_for_model(df, "text-ada-001")    

Error on classification. Retrying in 5 sec  Rate limit reached for default-global-with-image-limits in organization org-sjlda1axlW8Fd50AXi1ufZYl on requests per min. Limit: 60.000000 / min. Current: 70.000000 / min. Contact support@openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://beta.openai.com/account/billing to add a payment method.

Correct Prediction / Total Item: 3 / 37
Accuracy: 8.11%


## Custom Sentences: Neutral Performance

In [10]:
def evaluate_sentences(sentences, model):
    for sentence in sentences:
        label = classify_text(sentence, model=model)
        print(f"Text: {sentence}")
        print(f"Prediction: {label}\n")

In [11]:
sentences = [
    "J'ai adoré la vidéo, continuez comme ça. Je suis déjà abonné.",
    "La vidéo était mauvaise. Je ne l'aime pas.",
    "Les immigrés vivent avec nous dans notre pays.",
    "Je ne suis pas d'accord avec ce que vous dites. Je pense qu'une autre solution peut être trouvée."
]

### text-curie-001

In [12]:
evaluate_sentences(sentences, "text-curie-001")

Text: J'ai adoré la vidéo, continuez comme ça. Je suis déjà abonné.
Prediction: Toxic

Text: La vidéo était mauvaise. Je ne l'aime pas.
Prediction: Toxic

Text: Les immigrés vivent avec nous dans notre pays.
Prediction: Toxic

Text: Je ne suis pas d'accord avec ce que vous dites. Je pense qu'une autre solution peut être trouvée.
Prediction: Toxic



### text-ada-001

In [13]:
evaluate_sentences(sentences, "text-ada-001")

Text: J'ai adoré la vidéo, continuez comme ça. Je suis déjà abonné.
Prediction: Normal

Text: La vidéo était mauvaise. Je ne l'aime pas.
Prediction: Normal

Text: Les immigrés vivent avec nous dans notre pays.
Prediction: Normal

Text: Je ne suis pas d'accord avec ce que vous dites. Je pense qu'une autre solution peut être trouvée.
Prediction: Normal



### text-davinci-003

In [14]:
evaluate_sentences(sentences, "text-davinci-003")

Text: J'ai adoré la vidéo, continuez comme ça. Je suis déjà abonné.
Prediction: Normal

Text: La vidéo était mauvaise. Je ne l'aime pas.
Prediction: Normal

Text: Les immigrés vivent avec nous dans notre pays.
Prediction: Normal

Text: Je ne suis pas d'accord avec ce que vous dites. Je pense qu'une autre solution peut être trouvée.
Prediction: Normal



## Custom Sentences: Difficult Examples

In [15]:
sentences = [
    "Je pense que les immigrés ne devraient pas exister dans notre pays.",    
    "Je ne suis pas d'accord avec votre opinion, je pense que c'est ridicule.",
    "Je ne suis pas d'accord avec ton opinion, je pense que c'est ridicule, mais je te respecte quand même.",
    "Les hommes peuvent faire des travaux plus difficiles que les femmes.",
    "La présence des femmes dans le monde du travail est importante.",
    "Je n'ai pas du tout aimé la vidéo, mais bonne chance."
]

### text-curie-001

In [16]:
evaluate_sentences(sentences, "text-curie-001")

Text: Je pense que les immigrés ne devraient pas exister dans notre pays.
Prediction: Toxic

Text: Je ne suis pas d'accord avec votre opinion, je pense que c'est ridicule.
Prediction: Toxic

Text: Je ne suis pas d'accord avec ton opinion, je pense que c'est ridicule, mais je te respecte quand même.
Prediction: Toxic

Text: Les hommes peuvent faire des travaux plus difficiles que les femmes.
Prediction: Toxic

Text: La présence des femmes dans le monde du travail est importante.
Prediction: Toxic

Text: Je n'ai pas du tout aimé la vidéo, mais bonne chance.
Prediction: Toxic



### text-ada-001

In [17]:
evaluate_sentences(sentences, "text-ada-001")

Text: Je pense que les immigrés ne devraient pas exister dans notre pays.
Prediction: Normal

Text: Je ne suis pas d'accord avec votre opinion, je pense que c'est ridicule.
Prediction: Normal

Text: Je ne suis pas d'accord avec ton opinion, je pense que c'est ridicule, mais je te respecte quand même.
Prediction: Normal

Text: Les hommes peuvent faire des travaux plus difficiles que les femmes.
Prediction: Normal

Text: La présence des femmes dans le monde du travail est importante.
Prediction: Normal

Text: Je n'ai pas du tout aimé la vidéo, mais bonne chance.
Prediction: Normal



### text-davinci-001

In [18]:
evaluate_sentences(sentences, "text-davinci-003")

Text: Je pense que les immigrés ne devraient pas exister dans notre pays.
Prediction: Toxic

Text: Je ne suis pas d'accord avec votre opinion, je pense que c'est ridicule.
Prediction: Normal

Text: Je ne suis pas d'accord avec ton opinion, je pense que c'est ridicule, mais je te respecte quand même.
Prediction: Normal

Text: Les hommes peuvent faire des travaux plus difficiles que les femmes.
Prediction: Normal

Text: La présence des femmes dans le monde du travail est importante.
Prediction: Normal

Text: Je n'ai pas du tout aimé la vidéo, mais bonne chance.
Prediction: Normal

